# Quantum Error Correction Codes

## Introduction

Quantum error correction (QEC) is a fundamental pillar of fault-tolerant quantum computation. Unlike classical bits, which can be simply copied for redundancy, quantum states are subject to the **no-cloning theorem**, making error correction in quantum systems a nontrivial challenge. This notebook explores the theoretical foundations and practical implementation of quantum error correction codes.

## Theoretical Background

### Quantum Errors

A general single-qubit quantum state can be written as:

$$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$$

where $|\alpha|^2 + |\beta|^2 = 1$. Quantum errors can be decomposed into combinations of the Pauli operators:

$$I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}, \quad X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

- **Bit-flip error** ($X$): $|0\rangle \leftrightarrow |1\rangle$
- **Phase-flip error** ($Z$): $|+\rangle \leftrightarrow |-\rangle$
- **Bit-phase flip** ($Y = iXZ$): Combined error

### The 3-Qubit Bit-Flip Code

The simplest QEC code protects against bit-flip errors by encoding a single logical qubit into three physical qubits:

$$|0\rangle_L = |000\rangle, \quad |1\rangle_L = |111\rangle$$

A general state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ becomes:

$$|\psi\rangle_L = \alpha|000\rangle + \beta|111\rangle$$

### Syndrome Measurement

Error detection uses **stabilizer measurements** without collapsing the encoded quantum information. For the 3-qubit code, the stabilizer generators are:

$$g_1 = Z_1 Z_2 I_3, \quad g_2 = I_1 Z_2 Z_3$$

The syndrome $(s_1, s_2)$ identifies which qubit (if any) experienced an error:

| Syndrome | Error | Interpretation |
|----------|-------|----------------|
| $(0, 0)$ | $I$ | No error |
| $(1, 0)$ | $X_1$ | First qubit flipped |
| $(1, 1)$ | $X_2$ | Second qubit flipped |
| $(0, 1)$ | $X_3$ | Third qubit flipped |

### The 3-Qubit Phase-Flip Code

Similarly, phase-flip errors are corrected using the Hadamard-conjugated encoding:

$$|0\rangle_L = |{+}{+}{+}\rangle, \quad |1\rangle_L = |{-}{-}{-}\rangle$$

where $|\pm\rangle = \frac{1}{\sqrt{2}}(|0\rangle \pm |1\rangle)$.

### The 9-Qubit Shor Code

The **Shor code** concatenates bit-flip and phase-flip codes to protect against arbitrary single-qubit errors:

$$|0\rangle_L = \frac{1}{2\sqrt{2}}(|000\rangle + |111\rangle)^{\otimes 3}$$
$$|1\rangle_L = \frac{1}{2\sqrt{2}}(|000\rangle - |111\rangle)^{\otimes 3}$$

### Code Parameters: $[[n, k, d]]$

Quantum codes are characterized by:
- $n$: Number of physical qubits
- $k$: Number of encoded logical qubits
- $d$: Code distance (minimum weight of undetectable errors)

A code with distance $d$ can:
- **Detect** up to $d-1$ errors
- **Correct** up to $\lfloor(d-1)/2\rfloor$ errors

### Quantum Singleton Bound

The code parameters are constrained by:

$$n - k \geq 2(d - 1)$$

## Implementation

We will simulate quantum error correction codes by representing quantum states as vectors and applying operations as matrix multiplications. This classical simulation demonstrates the principles of QEC without requiring a quantum computer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

# Set random seed for reproducibility
np.random.seed(42)

# Define Pauli matrices
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)  # Hadamard

# Basis states
ket_0 = np.array([1, 0], dtype=complex)
ket_1 = np.array([0, 1], dtype=complex)

def tensor(*args):
    """Compute tensor product of multiple arrays."""
    result = args[0]
    for arr in args[1:]:
        result = np.kron(result, arr)
    return result

def normalize(state):
    """Normalize a quantum state."""
    return state / np.linalg.norm(state)

print("Pauli matrices defined successfully.")
print(f"X|0⟩ = |1⟩: {np.allclose(X @ ket_0, ket_1)}")
print(f"Z|0⟩ = |0⟩: {np.allclose(Z @ ket_0, ket_0)}")
print(f"Z|1⟩ = -|1⟩: {np.allclose(Z @ ket_1, -ket_1)}")

### 3-Qubit Bit-Flip Code Implementation

In [ ]:
class BitFlipCode:
    """Implementation of the 3-qubit bit-flip code."""
    
    def __init__(self):
        # Logical basis states
        self.logical_0 = tensor(ket_0, ket_0, ket_0)  # |000⟩
        self.logical_1 = tensor(ket_1, ket_1, ket_1)  # |111⟩
        
        # Stabilizer generators: Z₁Z₂ and Z₂Z₃
        self.stabilizer_1 = tensor(Z, Z, I)
        self.stabilizer_2 = tensor(I, Z, Z)
        
        # Error operators
        self.errors = {
            'I': tensor(I, I, I),
            'X1': tensor(X, I, I),
            'X2': tensor(I, X, I),
            'X3': tensor(I, I, X)
        }
        
        # Correction operators (same as errors for X errors)
        self.corrections = self.errors.copy()
    
    def encode(self, alpha, beta):
        """Encode a logical qubit |ψ⟩ = α|0⟩ + β|1⟩."""
        return alpha * self.logical_0 + beta * self.logical_1
    
    def measure_syndrome(self, state):
        """Measure the error syndrome.
        
        Returns (s1, s2) where each is 0 or 1.
        """
        # Expectation value gives +1 or -1
        # We convert to syndrome bits: +1 → 0, -1 → 1
        exp_1 = np.real(np.conj(state) @ self.stabilizer_1 @ state)
        exp_2 = np.real(np.conj(state) @ self.stabilizer_2 @ state)
        
        s1 = 0 if exp_1 > 0 else 1
        s2 = 0 if exp_2 > 0 else 1
        
        return (s1, s2)
    
    def syndrome_to_correction(self, syndrome):
        """Map syndrome to correction operator."""
        syndrome_map = {
            (0, 0): 'I',
            (1, 0): 'X1',
            (1, 1): 'X2',
            (0, 1): 'X3'
        }
        return syndrome_map[syndrome]
    
    def correct(self, state):
        """Detect and correct errors."""
        syndrome = self.measure_syndrome(state)
        correction_name = self.syndrome_to_correction(syndrome)
        correction_op = self.corrections[correction_name]
        corrected_state = correction_op @ state
        return corrected_state, syndrome, correction_name
    
    def decode(self, state):
        """Extract logical qubit amplitudes from encoded state."""
        alpha = np.vdot(self.logical_0, state)
        beta = np.vdot(self.logical_1, state)
        return alpha, beta

# Test the bit-flip code
code = BitFlipCode()

# Encode |+⟩ = (|0⟩ + |1⟩)/√2
alpha, beta = 1/np.sqrt(2), 1/np.sqrt(2)
encoded = code.encode(alpha, beta)

print("Testing 3-Qubit Bit-Flip Code")
print("="*40)
print(f"Original state: α={alpha:.4f}, β={beta:.4f}")
print(f"Encoded state norm: {np.linalg.norm(encoded):.4f}")

# Test each error and correction
for error_name, error_op in code.errors.items():
    corrupted = error_op @ encoded
    corrected, syndrome, correction = code.correct(corrupted)
    decoded_alpha, decoded_beta = code.decode(corrected)
    
    success = np.allclose([decoded_alpha, decoded_beta], [alpha, beta])
    print(f"\nError: {error_name}")
    print(f"  Syndrome: {syndrome}")
    print(f"  Correction: {correction}")
    print(f"  Recovery successful: {success}")

### 3-Qubit Phase-Flip Code Implementation

In [ ]:
class PhaseFlipCode:
    """Implementation of the 3-qubit phase-flip code."""
    
    def __init__(self):
        # Plus and minus states
        ket_plus = normalize(ket_0 + ket_1)
        ket_minus = normalize(ket_0 - ket_1)
        
        # Logical basis states in Hadamard basis
        self.logical_0 = tensor(ket_plus, ket_plus, ket_plus)  # |+++⟩
        self.logical_1 = tensor(ket_minus, ket_minus, ket_minus)  # |---⟩
        
        # Stabilizer generators: X₁X₂ and X₂X₃
        self.stabilizer_1 = tensor(X, X, I)
        self.stabilizer_2 = tensor(I, X, X)
        
        # Error operators (phase flips)
        self.errors = {
            'I': tensor(I, I, I),
            'Z1': tensor(Z, I, I),
            'Z2': tensor(I, Z, I),
            'Z3': tensor(I, I, Z)
        }
        
        self.corrections = self.errors.copy()
    
    def encode(self, alpha, beta):
        """Encode a logical qubit."""
        return alpha * self.logical_0 + beta * self.logical_1
    
    def measure_syndrome(self, state):
        """Measure the error syndrome."""
        exp_1 = np.real(np.conj(state) @ self.stabilizer_1 @ state)
        exp_2 = np.real(np.conj(state) @ self.stabilizer_2 @ state)
        
        s1 = 0 if exp_1 > 0 else 1
        s2 = 0 if exp_2 > 0 else 1
        
        return (s1, s2)
    
    def syndrome_to_correction(self, syndrome):
        """Map syndrome to correction operator."""
        syndrome_map = {
            (0, 0): 'I',
            (1, 0): 'Z1',
            (1, 1): 'Z2',
            (0, 1): 'Z3'
        }
        return syndrome_map[syndrome]
    
    def correct(self, state):
        """Detect and correct errors."""
        syndrome = self.measure_syndrome(state)
        correction_name = self.syndrome_to_correction(syndrome)
        correction_op = self.corrections[correction_name]
        corrected_state = correction_op @ state
        return corrected_state, syndrome, correction_name
    
    def decode(self, state):
        """Extract logical qubit amplitudes."""
        alpha = np.vdot(self.logical_0, state)
        beta = np.vdot(self.logical_1, state)
        return alpha, beta

# Test the phase-flip code
pcode = PhaseFlipCode()

alpha, beta = 1/np.sqrt(2), 1/np.sqrt(2)
encoded = pcode.encode(alpha, beta)

print("Testing 3-Qubit Phase-Flip Code")
print("="*40)
print(f"Original state: α={alpha:.4f}, β={beta:.4f}")

for error_name, error_op in pcode.errors.items():
    corrupted = error_op @ encoded
    corrected, syndrome, correction = pcode.correct(corrupted)
    decoded_alpha, decoded_beta = pcode.decode(corrected)
    
    success = np.allclose([np.abs(decoded_alpha), np.abs(decoded_beta)], 
                          [np.abs(alpha), np.abs(beta)])
    print(f"\nError: {error_name}")
    print(f"  Syndrome: {syndrome}")
    print(f"  Correction: {correction}")
    print(f"  Recovery successful: {success}")

### Error Rate Simulation

We now simulate the performance of the 3-qubit bit-flip code under varying error probabilities and compare it to an uncoded qubit.

In [ ]:
def simulate_bit_flip_code(p_error, n_trials=10000):
    """
    Simulate the 3-qubit bit-flip code under depolarizing-like bit-flip noise.
    
    Parameters:
    -----------
    p_error : float
        Probability of a bit-flip error on each physical qubit
    n_trials : int
        Number of Monte Carlo trials
    
    Returns:
    --------
    logical_error_rate : float
        Probability of logical error after correction
    """
    code = BitFlipCode()
    errors_occurred = 0
    
    # Random initial state for each trial
    for _ in range(n_trials):
        # Generate random state on Bloch sphere
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        alpha = np.cos(theta/2)
        beta = np.sin(theta/2) * np.exp(1j * phi)
        
        # Encode
        encoded = code.encode(alpha, beta)
        
        # Apply random bit-flip errors
        error_pattern = np.random.random(3) < p_error
        
        # Build error operator
        ops = [X if e else I for e in error_pattern]
        error_op = tensor(*ops)
        
        # Apply error
        corrupted = error_op @ encoded
        
        # Correct
        corrected, _, _ = code.correct(corrupted)
        
        # Decode and check fidelity
        decoded_alpha, decoded_beta = code.decode(corrected)
        
        # Check if logical error occurred (state not recovered)
        fidelity = np.abs(np.abs(decoded_alpha * np.conj(alpha)) + 
                         np.abs(decoded_beta * np.conj(beta)))**2
        
        if fidelity < 0.99:
            errors_occurred += 1
    
    return errors_occurred / n_trials

def uncoded_error_rate(p_error):
    """Error rate for a single uncoded qubit."""
    return p_error

def theoretical_coded_error_rate(p_error):
    """
    Theoretical logical error rate for 3-qubit code.
    
    Fails when 2 or more qubits flip:
    P_fail = 3p²(1-p) + p³ = 3p² - 2p³
    """
    return 3 * p_error**2 - 2 * p_error**3

# Run simulation
print("Running error rate simulation...")
p_values = np.linspace(0.001, 0.3, 30)
simulated_rates = []
theoretical_rates = []
uncoded_rates = []

for p in p_values:
    sim_rate = simulate_bit_flip_code(p, n_trials=5000)
    simulated_rates.append(sim_rate)
    theoretical_rates.append(theoretical_coded_error_rate(p))
    uncoded_rates.append(uncoded_error_rate(p))
    print(f"p={p:.3f}: Simulated={sim_rate:.4f}, Theory={theoretical_coded_error_rate(p):.4f}")

print("\nSimulation complete.")

### Visualization: Error Threshold

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Linear scale comparison
ax1 = axes[0]
ax1.plot(p_values, uncoded_rates, 'b--', label='Uncoded qubit', linewidth=2)
ax1.plot(p_values, theoretical_rates, 'r-', label='3-qubit code (theory)', linewidth=2)
ax1.scatter(p_values, simulated_rates, c='green', s=30, alpha=0.7, 
            label='3-qubit code (simulated)', zorder=5)

# Mark the threshold point where p = 3p² - 2p³
# Solving: p = 3p² - 2p³ → 1 = 3p - 2p² → 2p² - 3p + 1 = 0 → p = 0.5 or p = 1
# But physically meaningful threshold is where coded becomes better: p < 0.5
threshold = 0.5
ax1.axvline(x=threshold, color='purple', linestyle=':', alpha=0.7, 
            label=f'Threshold (p={threshold})')

ax1.set_xlabel('Physical error probability (p)', fontsize=12)
ax1.set_ylabel('Logical error probability', fontsize=12)
ax1.set_title('Quantum Error Correction: Error Threshold', fontsize=14)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 0.3])
ax1.set_ylim([0, 0.35])

# Add annotation
ax1.annotate('Coded is better\n(p < threshold)', 
             xy=(0.1, 0.05), fontsize=10, color='green',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# Plot 2: Log-log scale to see quadratic suppression
ax2 = axes[1]
ax2.loglog(p_values, uncoded_rates, 'b--', label='Uncoded qubit', linewidth=2)
ax2.loglog(p_values, theoretical_rates, 'r-', label='3-qubit code (theory)', linewidth=2)
ax2.scatter(p_values[np.array(simulated_rates) > 0], 
            np.array(simulated_rates)[np.array(simulated_rates) > 0], 
            c='green', s=30, alpha=0.7, label='3-qubit code (simulated)', zorder=5)

# Reference lines for scaling
p_ref = np.linspace(0.01, 0.1, 100)
ax2.loglog(p_ref, p_ref, 'k:', alpha=0.5, label='O(p) scaling')
ax2.loglog(p_ref, 3*p_ref**2, 'm:', alpha=0.5, label='O(p²) scaling')

ax2.set_xlabel('Physical error probability (p)', fontsize=12)
ax2.set_ylabel('Logical error probability', fontsize=12)
ax2.set_title('Error Suppression (Log-Log Scale)', fontsize=14)
ax2.legend(loc='lower right', fontsize=9)
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

### Analysis of the Shor Code

The 9-qubit Shor code provides protection against arbitrary single-qubit errors. Let's analyze its structure.

In [ ]:
class ShorCode:
    """Implementation of the 9-qubit Shor code.
    
    The Shor code concatenates:
    - Inner code: 3-qubit bit-flip code
    - Outer code: 3-qubit phase-flip code
    
    Logical states:
    |0⟩_L = (|000⟩ + |111⟩)(|000⟩ + |111⟩)(|000⟩ + |111⟩) / 2√2
    |1⟩_L = (|000⟩ - |111⟩)(|000⟩ - |111⟩)(|000⟩ - |111⟩) / 2√2
    """
    
    def __init__(self):
        # Build logical states
        plus_block = (tensor(ket_0, ket_0, ket_0) + tensor(ket_1, ket_1, ket_1)) / np.sqrt(2)
        minus_block = (tensor(ket_0, ket_0, ket_0) - tensor(ket_1, ket_1, ket_1)) / np.sqrt(2)
        
        self.logical_0 = tensor(plus_block, plus_block, plus_block)
        self.logical_1 = tensor(minus_block, minus_block, minus_block)
        
        # Normalize
        self.logical_0 = self.logical_0 / np.linalg.norm(self.logical_0)
        self.logical_1 = self.logical_1 / np.linalg.norm(self.logical_1)
        
        self.n_qubits = 9
    
    def encode(self, alpha, beta):
        """Encode a logical qubit."""
        return alpha * self.logical_0 + beta * self.logical_1
    
    def apply_single_qubit_error(self, state, qubit_idx, error_type='X'):
        """Apply a single-qubit error to the specified qubit."""
        error_ops = {'X': X, 'Y': Y, 'Z': Z, 'I': I}
        op = error_ops[error_type]
        
        # Build full error operator
        ops = [I if i != qubit_idx else op for i in range(self.n_qubits)]
        full_op = ops[0]
        for o in ops[1:]:
            full_op = np.kron(full_op, o)
        
        return full_op @ state
    
    def decode(self, state):
        """Extract logical qubit amplitudes."""
        alpha = np.vdot(self.logical_0, state)
        beta = np.vdot(self.logical_1, state)
        return alpha, beta

# Test Shor code
shor = ShorCode()

print("9-Qubit Shor Code Analysis")
print("="*40)
print(f"Code parameters: [[9, 1, 3]]")
print(f"  - 9 physical qubits")
print(f"  - 1 logical qubit")
print(f"  - Distance 3 (corrects 1 arbitrary error)")
print(f"\nLogical |0⟩ norm: {np.linalg.norm(shor.logical_0):.6f}")
print(f"Logical |1⟩ norm: {np.linalg.norm(shor.logical_1):.6f}")
print(f"Orthogonality ⟨0|1⟩: {np.abs(np.vdot(shor.logical_0, shor.logical_1)):.6f}")

# Analyze logical states
print(f"\nState vector dimension: 2^9 = {2**9}")

### Fidelity Analysis Under Different Error Types

In [ ]:
def compute_fidelity(state1, state2):
    """Compute fidelity between two pure states."""
    return np.abs(np.vdot(state1, state2))**2

# Test Shor code against various single-qubit errors
alpha, beta = 1/np.sqrt(2), 1/np.sqrt(2)
encoded = shor.encode(alpha, beta)

print("Shor Code Error Analysis")
print("="*50)
print(f"Initial state: |+⟩_L = (|0⟩_L + |1⟩_L)/√2")
print("\nSingle-qubit errors (before correction):")

error_fidelities = {'X': [], 'Y': [], 'Z': []}

for error_type in ['X', 'Y', 'Z']:
    print(f"\n{error_type} errors:")
    for qubit in range(9):
        corrupted = shor.apply_single_qubit_error(encoded, qubit, error_type)
        fidelity = compute_fidelity(encoded, corrupted)
        error_fidelities[error_type].append(fidelity)
        
    avg_fidelity = np.mean(error_fidelities[error_type])
    print(f"  Average fidelity with original: {avg_fidelity:.4f}")

# Visualize error impact
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(9)
width = 0.25

ax.bar(x - width, error_fidelities['X'], width, label='X (bit-flip)', color='red', alpha=0.7)
ax.bar(x, error_fidelities['Z'], width, label='Z (phase-flip)', color='blue', alpha=0.7)
ax.bar(x + width, error_fidelities['Y'], width, label='Y (both)', color='purple', alpha=0.7)

ax.set_xlabel('Qubit Index', fontsize=12)
ax.set_ylabel('Fidelity with Encoded State', fontsize=12)
ax.set_title('Impact of Single-Qubit Errors on Shor-Encoded State', fontsize=14)
ax.set_xticks(x)
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(True, alpha=0.3)

# Add block separators
for i in [2.5, 5.5]:
    ax.axvline(x=i, color='gray', linestyle='--', alpha=0.5)

ax.text(1, 1.02, 'Block 1', ha='center', fontsize=10)
ax.text(4, 1.02, 'Block 2', ha='center', fontsize=10)
ax.text(7, 1.02, 'Block 3', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## Summary and Key Results

### Main Findings

1. **Error Threshold**: The 3-qubit code provides benefit when $p < 0.5$, where $p$ is the single-qubit error probability.

2. **Quadratic Suppression**: The logical error rate scales as $O(p^2)$ compared to $O(p)$ for uncoded qubits.

3. **Concatenation**: The Shor code combines bit-flip and phase-flip protection, achieving arbitrary error correction.

### Theoretical Significance

The **threshold theorem** states that if physical error rates are below a threshold $p_{th}$, arbitrarily long quantum computations become possible through concatenated coding. Modern surface codes achieve thresholds around $p_{th} \approx 1\%$.

### Code Comparison

| Code | $[[n, k, d]]$ | Corrects | Overhead |
|------|---------------|----------|----------|
| 3-qubit bit-flip | [[3,1,1]] | 1 X error | 3× |
| 3-qubit phase-flip | [[3,1,1]] | 1 Z error | 3× |
| Shor code | [[9,1,3]] | 1 arbitrary | 9× |
| Steane code | [[7,1,3]] | 1 arbitrary | 7× |
| Surface code | [[O(d²),1,d]] | $(d-1)/2$ | $d^2$× |

In [ ]:
# Final verification: ensure plot.png exists
import os
if os.path.exists('plot.png'):
    print("✓ Verification artifact 'plot.png' successfully created.")
    print(f"  File size: {os.path.getsize('plot.png')} bytes")
else:
    print("✗ Warning: plot.png was not created!")